In [28]:
import os
import json
import re
from operator import itemgetter

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import words
from nltk.corpus import wordnet as wn
from nltk.corpus import framenet as fn
from nltk.corpus import stopwords
from nltk.corpus.reader.framenet import PrettyList
from nltk.stem.snowball import SnowballStemmer
from time import time
# ...
# nltk.download('words')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')

In [29]:
directory_summary_main = r"data/BBC News Summary/News Articles/"
sub_dirs = os.listdir(directory_summary_main)

word_count_dict = {}
root_word_count_dict = {}

total_token_count = 0
max_file_count = 1000

i = 0

for sub in sub_dirs:
    files = os.listdir(directory_summary_main + sub)
    for f in files:
        try:
            text = open(directory_summary_main + sub + "/" + f, "r").read()
            tokens = word_tokenize(text.lower())
            total_token_count = total_token_count + len(tokens)
            for tok in tokens:
                if tok.isalpha():
                    if tok not in word_count_dict:
                        word_count_dict[tok] = 0
                    word_count_dict[tok] = word_count_dict[tok] + 1
            i = i + 1
        except:
            continue

        #     if i % 1000==1:
        #         print(i)
        #     if i > max_file_count:
        #         break

word_freq_dict = {}
for w in word_count_dict:
    word_freq_dict[w] = word_count_dict[w]/total_token_count

word_freq_dict_val = {k: v for k, v in sorted(word_freq_dict.items(), key=lambda x: x[1], reverse= False)}  

with open("word_fequency.json", "w") as outfile:
    json.dump(word_freq_dict_val, outfile)
    
# word_freq_dict_val

In [66]:
def get_event_typing(w, thres = 0.0, m = 3, depth = 3):
    ws = wn.synsets(w)
    
    hyper = lambda s: s.hypernyms()
    types1 = {}
    types2 = {}

    for e in ws:
        w_tmp = e.name().split(".")[0]
        if w_tmp==w:
            t = 0
            c = 0 
            i = 0
            tmp_types = ""
            
            tmp_event_types = list(e.closure(hyper))
            
            for tmp_event in tmp_event_types:
                d = wn.path_similarity(e, tmp_event)
                tmp_event_name = tmp_event.name().split(".")[0]
                
                b = m**i
                t = t + (d/b)
                types2[tmp_event_name] = d
                
                if d and (d >= (thres/t)):
                    tmp_types = tmp_types + ":" + tmp_event_name
                    if c >= depth - 1:
                        break
                    c = c + 1
                    
                i = i + 1
                
            if len(tmp_types) > 0:
                types1[tmp_types] = t
                
    types1 = {k: v for k, v in sorted(types1.items(), key=lambda item: item[1], reverse=True)}
    types2 = {k: v for k, v in sorted(types2.items(), key=lambda item: item[1], reverse=True)}
    
    return types1, types2


def get_event_typing_main(w, pos, thres = 0.0, m = 3, depth = 3):
    if not pos:
        pos = nltk.tag.pos_tag(w.split())[0][1]
    
    types1, types2 = get_event_typing(w, thres, m, depth)
    
    if not (types1 or types2):
        if pos.startswith("V"):    
            w_tmp = WordNetLemmatizer().lemmatize(w, "v")
            types1, types2 = get_event_typing(w_tmp, thres, m, depth)
            if (types1 or types2):
                return types1, types2
            
        if pos.startswith("N"):
            w_tmp = WordNetLemmatizer().lemmatize(w, "n")
            types1, types2 = get_event_typing(w_tmp, thres, m, depth)
            if (types1 or types2):
                return types1, types2
            
        w_tmp = WordNetLemmatizer().lemmatize(w)
        types1, types2 = get_event_typing(w_tmp, thres, m, depth)
        
        if (types1 or types2):
            return types1, types2
        
        stemmer = SnowballStemmer("english")
        w_tmp = stemmer.stem(w)
        types1, types2 = get_event_typing(w_tmp, thres, m, depth)
            
    
    return types1, types2


def replacer(match):
    if match.group(1) is not None:
        return '{} '.format(match.group(1))
    else:
        return ' {}'.format(match.group(2))

def preprocess_text(text):
    rx = re.compile(r'^(\W+)|(\W+)$')
    text = " ".join([rx.sub(replacer, word) for word in raw_text.split()])
    text = re.sub("'", " ' ", text)
    text = re.sub("\t+", " ", text)
    text = re.sub("\s+", " ", text)
    text = re.sub("\n+", " ", text)
    
    return text

In [76]:
raw_text = '''“If the coffee tastes bitter, it is bitter. It’s not your tongue malfunctioning.” –E. Xu

Ugh. You’ve awakened feeling sick, with an array of frightening symptoms—perhaps a strange rash, a headache, stomach pain, and shortness of breath. You feel awful, which makes you feel vulnerable. You need help; so, you call the doctor and make an appointment. Hope is the motivation that gets you out of bed and dressed. Hope drags you out the front door and into the physician’s waiting room. What, then, is the impact when the doctor you’ve come to for help brushes off your physical distress as anxiety, depression, or the extra weight you put on during the pandemic?

The essence of dehumanization is the denial of another person’s reality. To be told, in the absence of clinical evidence, that one’s pain is not real or that one’s biological symptoms are exaggerated or psychosomatic, is medical gaslighting. Medical gaslighting has been well-documented across a variety of clinical conditions. It is extremely commonplace for women, and even more prevalent for women of color, disabled women, LGBTQ individuals, older women, and women of lower socioeconomic status (Moyer, 2022). Medical gaslighting drastically increases morbidity and mortality, exacerbates symptom distress, and imposes self-doubt, confusion, humiliation, and, ironically, anxiety and depression. Medical gaslighting erodes trust in the doctor-patient relationship and interferes with help-seeking for legitimate medical issues.

One problem is that much less is known about women’s bodies than men’s. In the long history of the world, women have been excluded from scientific research even though there are male-female biological differences at the cellular level (Moyer, 2022). Women have only been included in medical trials since the 1990s. The assumption that the male can serve as the representative of the species, has left us with a health care system made by men, for men. For example, while 90% of women report at least one PMS symptom, or female trouble, five times more studies have researched erectile dysfunction, affecting just 19% of males.

Women are more likely to be misdiagnosed, and twice as likely as men to be given a mental health diagnosis, even when they have identical symptoms and they’re suggestive of a biological condition (Oladele, et al., 2022). Compared with men, women face longer wait times for cancer or heart disease diagnoses, and are seven times more likely to be discharged from the ER in the midst of a heart attack. Women are treated less aggressively for chronic pain, traumatic brain injury, and autoimmune issues, and must wait longer for pain medication when experiencing severe distress (Moyer, 2022).

Gaslighting has occurred for centuries in relation to women’s health concerns. The term "hysteria" is derived from the Greek word for uterus, reflecting the belief that hysteria was caused by a defective womb. The oldest record of hysteria dates back to 1900 B.C. when Egyptians recorded behavioral abnormalities in adult women on medical papyrus (Micale, 2019). By the turn of the 20th century, hysteria had been codified as a mental disorder, setting the precedent for modern-day women to be told their sicknesses are in their heads.

Another factor is the dysfunctional U.S. medical system. Doctors are strained with large patient loads and must complete services in brief appointments. Insufficient time may leave doctors feeling they do not have the capacity to actually listen to their patient’s concerns. Further, the hyper-specialization of Western medicine easily leads to a diffusion of responsibility. Finally, prevailing medical training minimizes women’s issues and gives little attention to hard-to-identify syndromes such as chronic pain, endometriosis, autoimmune disorders, and interstitial cystitis. Medical gaslighting can be unconscious, non-intentional, and without malice, but it is still reflective of pervasive, systemic gender bias and the impact remains brutal.

Awareness increases opportunities for patient empowerment. Medical gaslighting includes misdiagnoses, denial of needed tests, medications, and treatments, dismissal from clinical care settings, and being blamed for not trying hard enough to recover (Castanares-Zapatero, et al., 2021). Subtle forms of gaslighting include condescending comments, interrupting or disregarding the client’s comments and point of view, dismissive gestures and expressions (e.g., a shrug, sigh, silence, or raised eyebrow), countering or questioning someone’s memory or attribution of events, passive invalidation, trivializing, forgetting, withholding, and diverting (e.g., changing the subject). A person should be skeptical of their medical care when they feel ignored, minimized, or dismissed, or when diagnosed with depression or anxiety without exhibiting the emotional cognitive symptoms associated with the disorder or in the absence of a thorough workup to consider underlying causes.

Listen to your body’s wisdom and believe in your internal voice. If you feel you’ve experienced gaslighting, speak up. If this proves difficult, try bringing a friend along as a medical advocate. If you experience medical gaslighting and the doctor is unresponsive when you explain the symptoms are not normal for you, do not return to that setting. Whenever possible, prepare for a doctor’s appointment in advance. Keep a log of symptoms, triggers, exacerbating circumstances, and what may or may not be helping. A second opinion is always your prerogative.

You are the expert regarding your own body. You have the right to be heard, acknowledged, and believed. Good health care is integrated, patient-centered, personalized, and participatory. When the PCP and specialists work in partnership with the patient, better medical outcomes, satisfaction, and reduced costs are achieved (Reiss-Brennan et al., 2016).

'''

In [78]:
stops = set(stopwords.words('english'))
upper_freq_noun = 0.0001
upper_freq_verb = 0.00001



raw_text = "The number of people already infected by the mystery virus emerging in China is far greater than official figures suggest, scientists have told the BBC. There have been more than 60 confirmed cases of the new coronavirus, but UK experts estimate a figure nearer 1700. Two people are known to have died from the respiratory illness, which appeared in Wuhan city in December."

start_time = time()
text = preprocess_text(raw_text)

# print(text)
tokens = text.lower().split()

pos_tags = nltk.tag.pos_tag(text.split())
for x in pos_tags:
    token = x[0].lower()
    pos = x[1]
    if (token not in stops) and ((pos.startswith("V")) or (pos=="NN")) and (token in word_freq_dict_val):
        if (pos.startswith("V")) and (word_freq_dict_val[token] > upper_freq_verb):
            continue
        if (pos=="NN") and (word_freq_dict_val[token] > upper_freq_noun):
            continue
        a, b = get_event_typing_main(token, pos=pos, thres=0.0, m=10, depth=3) 
#         if a or b:
#             print("_"*100)
#             print(token, end=", ")
#             print(token, pos)
#         if a:
#             print(a)
#             print("\n")
#         if b:
#             print(b)
        
print("Processing Time: ", time() - start_time)

Processing Time:  0.00464177131652832


In [56]:
word_freq_dict_val["ash"]

6.163847391357053e-06